# Проект для "Викишоп" с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# Подготовка

In [108]:
import pandas as pd
import numpy as np
import torch
!pip install transformers
import transformers as ppb
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
!pip install pytorch-pretrained-bert
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
!ls 'drive/MyDrive/toxic_comments.csv'


In [109]:

data = pd.read_csv('/content/toxic_comments.csv')

Изучим данные:

In [4]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data.tail()

,text,toxic
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0
159570,"""\nAnd ... I really don't think you understand...",0


In [6]:
data.sample(5)

,text,toxic
133111,I is pissed off with Zzzz,0
94822,"Sorry, but Wikipedia is not a place to write a...",0
56648,"""\nI agree, Opabinia's most recent suggestion ...",0
108363,You are being incredibly dishonest by arguing ...,0
135192,Time Times (2008-05),0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [8]:
data.duplicated().sum()

0

In [9]:
data.isnull().sum()

text     0
toxic    0
dtype: int64

**Вывод:** Были изучены текстовые данные - пропусков и дубликатов не обнаружено

Для удобства взяла в работу лишь часть данных (Colab вылетал на одном и том же моменте обработки данных)

In [309]:
batch_1 = data[:2600]
batch_1['clear_text'] = batch_1['text'].str.replace('[^a-zA-Z]', ' ').str.lower()

In [11]:
!pip install ppb

In [310]:

model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


In [162]:
batch_1['clear_text']

0       explanation why the edits made under my userna...
1       d aww  he matches this background colour i m s...
2       hey man  i m really not trying to edit war  it...
3         more i can t make any real suggestions on im...
4       you  sir  are my hero  any chance you remember...
                              ...                        
2995    new section at wp ani  there is now a new sect...
2996                     and asking top stop involving me
2997    re  all items i know that you said i did somet...
2998      so you not going tell me why you created so ...
2999    john phillip key  born   august       in auckl...
Name: clear_text, Length: 3000, dtype: object

In [311]:
tokenized = batch_1['clear_text'].apply((lambda x: tokenizer.encode(x, max_length = 165, truncation=True, add_special_tokens=True)))

In [313]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [314]:
np.array(padded).shape

(2600, 165)

In [315]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2600, 165)

In [316]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

In [317]:
%%time
batch_size = 2 # для примера возьмем такой батч, где будет всего две строки датасета
embeddings = [] 
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]).cuda() # здесь срез общих данных делаем
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
 
        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
 
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
        del batch
        del attention_mask_batch
        del batch_embeddings
 
features = np.concatenate(embeddings) # наши эмбеддинги соединяются в признаки


CPU times: user 22.9 s, sys: 7.21 s, total: 30.1 s
Wall time: 30 s


**Вывод:** с помощью модели BERT трансформировали текстовые данные в удобный нам формат

# 2. Обучение

In [318]:
X = features


**Обучим логистическую модель:**

In [319]:
y = batch_1['toxic']

In [320]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.25, random_state=42)

In [321]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(X_train, y_train)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.9320512820512821


In [322]:
predicted_valid = grid_search.predict(X_valid)


In [323]:
print(f1_score(y_valid, predicted_valid))


0.64


In [324]:
predicted_test = grid_search.predict(X_test)

In [325]:
print(f1_score(y_test, predicted_test))

0.7567567567567567


**Обучим дерево решений:**

In [326]:
random_forest_clf = RandomForestClassifier(max_depth=2, random_state=0)
random_forest_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [327]:
predicted_valid = random_forest_clf.predict(X_valid)


In [328]:
print(f1_score(y_valid, predicted_valid))

0.0


In [329]:
predicted_test = random_forest_clf.predict(X_test)


In [330]:
print(f1_score(y_test, predicted_test))

0.0


Обучим модель LGBMClassifier


In [331]:
import lightgbm as ltb

model = ltb.LGBMClassifier()
model.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [332]:
predicted_valid = model.predict(X_valid)
print(f1_score(y_valid, predicted_valid))

0.5569620253164557


In [333]:
predicted_test = model.predict(X_test)
print(f1_score(y_test, predicted_test))

0.6117647058823529


Обучим модель классификации Catboost

In [336]:
!pip install catboost
from catboost import CatBoostClassifier


model = CatBoostClassifier(iterations=1000,
                           devices='0:1')
model.fit(X_train,
          y_train,
          verbose=100)

Learning rate set to 0.012457
0:	learn: 0.6807189	total: 219ms	remaining: 3m 38s
100:	learn: 0.2093553	total: 18.9s	remaining: 2m 47s
200:	learn: 0.1282338	total: 37.4s	remaining: 2m 28s
300:	learn: 0.0912414	total: 56.1s	remaining: 2m 10s
400:	learn: 0.0690892	total: 1m 14s	remaining: 1m 51s
500:	learn: 0.0537681	total: 1m 33s	remaining: 1m 32s
600:	learn: 0.0419985	total: 1m 52s	remaining: 1m 14s
700:	learn: 0.0333448	total: 2m 11s	remaining: 55.9s
800:	learn: 0.0262713	total: 2m 29s	remaining: 37.2s
900:	learn: 0.0208893	total: 2m 48s	remaining: 18.5s
999:	learn: 0.0171253	total: 3m 6s	remaining: 0us


In [337]:
predicted_valid = model.predict(X_valid)
print(f1_score(y_valid, predicted_valid))

0.5194805194805194


In [338]:
predicted_test = model.predict(X_test)
print(f1_score(y_test, predicted_test))

0.5714285714285715


# 3. Выводы

На входе нам были даны текстовые данные, мы их с помощью модели BERT преобразовали в векторы и с помощью логистической модели с помощью подбора гиперапараметров добились f1 более 0.75.
Опытным путем выяснили, что на качество модели больше всего влияет количество признаков в датасете, чистота исходного текста, длина токенайзера. В тоже время, будучи ограниченными в ресурсах, нам помогло то, что мы взяли не всю выборку и разбили ее на батчи. Это позволило экспериментировать с размером выборки.

Случайный лес и бустинговые модели показали не такой хороший результат, случайный лес как оказалось восве непригоден для таких задач или требует более детального подбора параметров, в том числе и выбора версии BERT.